<a href="https://colab.research.google.com/github/the-original-shruti/real-time-tradebot/blob/master/tradebot/RealTimeTradingBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Dependencies

In [2]:
# check java version >=8
!java -version

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [3]:
# install es server
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [4]:
# install kafka
!wget https://mirrors.estointernet.in/apache/kafka/2.6.0/kafka_2.13-2.6.0.tgz
!tar -xzf kafka_2.13-2.6.0.tgz

--2020-10-15 12:51:31--  https://mirrors.estointernet.in/apache/kafka/2.6.0/kafka_2.13-2.6.0.tgz
Resolving mirrors.estointernet.in (mirrors.estointernet.in)... 43.255.166.254, 2403:8940:3:1::f
Connecting to mirrors.estointernet.in (mirrors.estointernet.in)|43.255.166.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65537909 (63M) [application/octet-stream]
Saving to: ‘kafka_2.13-2.6.0.tgz’

kafka_2.13-2.6.0.tg 100%[===================>]  62.50M  6.23MB/s    in 13s     

2020-10-15 12:51:45 (4.94 MB/s) - ‘kafka_2.13-2.6.0.tgz’ saved [65537909/65537909]



In [5]:
!pip install websocket-client
!pip install faust
!pip install elasticsearch
!pip install geminipy 

     |████████████████████████████████| 204kB 7.5MB/s 
     |████████████████████████████████| 337kB 9.0MB/s 
     |████████████████████████████████| 143kB 19.0MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
     |████████████████████████████████| 1.2MB 17.2MB/s 
     |████████████████████████████████| 296kB 71.7MB/s 
     |████████████████████████████████| 266kB 61.4MB/s 
     |████████████████████████████████| 153kB 63.4MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
  Created wheel for opentracing: filename=opentracing-1.3.0-cp36-none-any.whl size=18449 sha256=55b071f69e0386911fe62be94e32d8230f1a05795052592fb71f0dff11843c6b
  Stored in directory: /root/.cache/pip/wheels/5a/c7/a8/55354b8928b16dafbcd73fb8fc33d7d55cf61ed0845b912ec8
  Created wheel for colorclass: filename=colorclass-2.2.0-cp36-none-any.whl size=19397 sha256=2837889406e9411c14043e033ebeb8b5a7463ff5787d4dd6c05d3df4112671bd
  Stored in directory: /root/.cache/pip/wheels/d1/86/9d/16127127306a92d7

In [25]:
# download the gemini.py module which contains the definition of faust app from our github.
!wget https://raw.githubusercontent.com/the-original-shruti/real-time-tradebot/master/tradebot/gemini.py

--2020-10-15 12:56:20--  https://raw.githubusercontent.com/the-original-shruti/real-time-tradebot/master/tradebot/gemini.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1616 (1.6K) [text/plain]
Saving to: ‘gemini.py’

gemini.py           100%[===================>]   1.58K  --.-KB/s    in 0s      

2020-10-15 12:56:20 (43.9 MB/s) - ‘gemini.py’ saved [1616/1616]



In [ ]:
# upload forecast csv file and api_details.json file
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Start ElasticSearch Server 

In [1]:
import os
from subprocess import Popen, PIPE, STDOUT

In [63]:
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
!echo "Waiting for 15 secs until elasticsearch is up and running"
!sleep 15

Waiting for 15 secs until elasticsearch is up and running


In [65]:
# check server is running
!curl -X GET "localhost:9200/"

{
  "name" : "b0ce72e4e73c",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "lH16qYDlRjOKWtfIPaKdoA",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# Start Kafka Server

In [11]:
!echo "Waiting for 20 secs until kafka and zookeeper services are up and running"
!cd kafka_2.13-2.6.0 && bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
!sleep 10
!cd kafka_2.13-2.6.0 && bin/kafka-server-start.sh -daemon config/server.properties
!sleep 10

Waiting for 20 secs until kafka and zookeeper services are up and running


In [12]:
# check kafka is running 
!ps -ef | grep kafka

root        2366       1  1 13:29 ?        00:00:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-2.6.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=102400 -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-2.6.0/bin/../logs -Dlog4j.configuration=file:bin/../config/log4j.properties -cp /content/kafka_2.13-2.6.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-2.6.0/bin/../libs/aopalliance-repackaged-2.5.0.jar:/content/kafka_2.13-2.6.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-2.6.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-2.6.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-2.6.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13-2.6.0/bin/../lib

# Start Gemini Feed

In [13]:
from kafka import KafkaProducer, KafkaConsumer
import websocket
import json
import threading

In [14]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

In [15]:
def on_message(ws, message):
    message = json.loads(message)
    if message['type'] == 'update':
        for i in message['events']:
            if 'side' in i:
                payload = {'side': i['side'], 'price': i['price'], 'remaining': i['remaining']}
                sent = producer.send(topic='gemini-feed', value=bytes(json.dumps(payload), 'utf-8'))
        producer.flush()

In [16]:
ws = websocket.WebSocketApp("wss://api.gemini.com/v1/marketdata/BTCUSD",
                            on_message=on_message)

In [17]:
ws_thread = threading.Thread(target=ws.run_forever)
ws_thread.setDaemon(True)
ws_thread.start() # this will run as a daemon process

# Faust Streaming

In [56]:
!faust -A gemini -l info worker --web-port=6066

(0l(BƒaµS† v1.10.4(0wqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqk(B
(0x(B id          (0x(B gemini                                   (0x(B
(0x(B transport   (0x(B [URL('kafka://localhost:9092')]          (0x(B
(0x(B store       (0x(B memory:                                  (0x(B
(0x(B web         (0x(B http://localhost:6066/                   (0x(B
(0x(B log         (0x(B -stderr- (info)                          (0x(B
(0x(B pid         (0x(B 4661                                     (0x(B
(0x(B hostname    (0x(B b0ce72e4e73c                             (0x(B
(0x(B platform    (0x(B CPython 3.6.9 (Linux x86_64)             (0x(B
(0x(B drivers     (0x(B                                          (0x(B
(0x(B   transport (0x(B aiokafka=1.1.6                           (0x(B
(0x(B   web       (0x(B aiohttp=3.6.3                            (0x(B
(0x(B datadir     (0x(B /content/gemini-data                     (0x(B
(0x

In [ ]:
# faust_worker = Popen(['faust -A gemini -l info worker --web-port=6066'], 
#                   stdout=PIPE, stderr=STDOUT,
#                   preexec_fn=lambda: os.setuid(5)  # as daemon
#                  )
# !echo "Waiting for 15 secs until faust worker is up and running"
# !sleep 15

# Gemini Trading Bot

In [19]:
import pandas as pd
from datetime import datetime, timedelta
from geminipy import Geminipy
import requests
from elasticsearch import Elasticsearch

In [20]:
# load in csv of forecasts
fcst = pd.read_csv('2020_full_trade_forecast.csv')

# generate a datetime field from the string timestamp `ds`
fcst['timestamp'] = fcst.ds.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# set the index as the timestamp so we can easily make lookups based on time
fcst = fcst.set_index('timestamp')[['side']]

In [59]:
es = Elasticsearch()
if not es.indices.exists("gemini"):
    es.indices.create(index='gemini')

In [41]:
consumer = KafkaConsumer('faust.out',
                         bootstrap_servers=['localhost:9092'],
                        auto_offset_reset='earliest',
                        enable_auto_commit=True,
                        value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [26]:
# trade 50% of our account balance
trade_pct = 0.5

# require that price_volume threshold be between 0.1 and 2
# this indicates that sell side volume is 1/10th and double that of buy side
# suggesting (relatively) normal liquidity thresholds.
# if sell side liquidity is above 2x that of the buy side we might think that 
# price may not move in the next minute.
threshold = (0.1, 2)

In [27]:
with open('api_details.json') as f:
    api_details = json.load(f)
    gemini_api_key = api_details['api-key']
    gemini_api_secret = api_details['secret']

In [28]:
con = Geminipy(api_key=gemini_api_key, secret_key=gemini_api_secret, live=False)

In [29]:
def get_btc_balance(con):
    for i in con.balances().json():
        if i['currency'] == 'BTC':
            i['amount'] = float(i['amount'])
            i['available'] = float(i['available'])
            i['availableForWithdrawal'] = float(i['availableForWithdrawal'])
            i['doc_type'] = 'balance'
            return i
        
get_btc_balance(con)

{'amount': 895.58703,
 'available': 895.58703,
 'availableForWithdrawal': 895.58703,
 'currency': 'BTC',
 'doc_type': 'balance',
 'type': 'exchange'}

In [30]:
def get_usd_balance(con):
    for i in con.balances().json():
        if i['currency'] == 'USD':
            i['amount'] = float(i['amount'])
            i['available'] = float(i['available'])
            i['availableForWithdrawal'] = float(i['availableForWithdrawal'])
            i['doc_type'] = 'balance'
            return i
        
get_usd_balance(con)

{'amount': 1283618.7842220885,
 'available': 1283618.78,
 'availableForWithdrawal': 1283618.78,
 'currency': 'USD',
 'doc_type': 'balance',
 'type': 'exchange'}

In [31]:
def get_ticker(con):
    '''
    NOTE: We need to ensure that numbers are numbers, not strings, for ES.
    Otherwise we would need to specify a mapping.
    '''
    ticker = con.pubticker().json()
    ticker['ask'] = float(ticker['ask'])
    ticker['bid'] = float(ticker['bid'])
    ticker['last'] = float(ticker['last'])
    ticker['volume_BTC'] = float(ticker['volume'].pop('BTC'))
    ticker['volume_USD'] = float(ticker['volume'].pop('USD'))
    ticker['timestamp'] = datetime.fromtimestamp(ticker['volume'].pop('timestamp')/1000)
    ticker.pop('volume')
    ticker['doc_type'] = 'ticker'
    return ticker

get_ticker(con)

{'ask': 11386.72,
 'bid': 11383.38,
 'doc_type': 'ticker',
 'last': 11389.66,
 'timestamp': datetime.datetime(2020, 10, 15, 13, 35),
 'volume_BTC': 1098.9609424,
 'volume_USD': 12503655.790815365}

In [32]:
def format_order(order):
    '''
    NOTE: We need to ensure that numbers are numbers, not strings, for ES.
    Otherwise we would need to specify a mapping.
    
    Also, we convert epoch time to Python datetime which is natively recognized as a time field by ES.
    Epoch time, without a custom mapping, would appear as a number.
    '''
    order_dict = order.json()
    try:
        order_dict['timestamp'] = datetime.fromtimestamp(int(order_dict['timestamp']))
    except:
        # no timestamp field, try timestampms
        try:
            order_dict['timestamp'] = datetime.fromtimestamp(int(order_dict['timestampms']))
        except:
            # no timestampms, set to now
            order_dict['timestamp'] = datetime.now()
    order_dict['price'] = float(order_dict['price'])
    order_dict['original_amount'] = float(order_dict['original_amount'])
    order_dict['remaining_amount'] = float(order_dict['remaining_amount'])
    order_dict['avg_execution_price'] = float(order_dict['avg_execution_price'])
    order_dict['executed_amount'] = float(order_dict['executed_amount'])
    order_dict['doc_type'] = 'order'
    return order_dict

In [33]:
def make_order(con, amount, side, ticker):
    # if we are buying we should take the last ask price
    if side == 'buy':
        bid_ask = 'ask'

    # if we are selling we should take the last bid price        
    elif side == 'sell':
        bid_ask = 'bid'
        
    order = con.new_order(amount = amount, # set order amount
                      price = ticker[bid_ask], # grab latest bid or ask price
                      side = side, # set side (either buy/sell)
                      options = ['immediate-or-cancel'] # take liquidity with an immediate trade
                     )
    # format order for Elasticsearch
    order_dict = format_order(order)
    
    return order_dict

In [34]:
def lookup_side(fcst):
    # get current timestamp
    now = datetime.now()
    
    # we must add 1 minute to the time we lookup
    # this is because our forecasts are for whether we should have bought/sold in a given minute
    # so, we want trade with our prediction in mind (hence, add 1 minute)
    ts_prior = datetime(year=now.year, month=now.month, day=now.day, hour=now.hour, minute=now.minute)
    ts = ts_prior + timedelta(minutes=1)
    
    last_fcst = fcst.loc[ts_prior].side
    curr_fcst = fcst.loc[ts].side
    return {'last_fcst': last_fcst, 'side': curr_fcst} # return looked up side (either buy or sell)

In [35]:
def get_last_buy(es):
    query = {
      "sort" : [
            { "timestamp" : {"order" : "desc"}}
        ],
      "query": {
        "bool": {
          "must": [
            {
              "match_phrase": {
                "side": {
                  "query": "buy"
                }
              }
            }
          ]
        }
      }
    }

    results = es.search(index='gemini', doc_type='gem', body=query)
    last_buy = results['hits'][0]
    last_price = last_buy['_source']['price']
    return last_price

In [36]:
def score_trade(es, order, balance):
    last_price = get_last_buy(es)
    
    # get timestamp 
    now = datetime.now()
    timestamp = datetime(year=now.year, month=now.month, day=now.day, hour=now.hour, minute=now.minute)
    
    # grab sell order price
    curr_price = order['price']
    profit = (curr_price - last_price) * balance
    
    return {'profit': profit, 'timestamp': timestamp, 'doc_type': 'score'}

In [51]:
def process_msg(message, fcst, con, es, threshold, trade_pct, last_price, traded):
    msg = message.value
    
    # convert msg epoch time to datetime
    msg['timestamp'] = datetime.fromtimestamp(msg['timestamp'])
    
    # get ticker
    ticker = get_ticker(con)

    # check side from model
    lookup = lookup_side(fcst)
    side = lookup['side']
    last_fcst = lookup['last_fcst']

    msg['doc_type'] = msg.pop('type')

    # index msg to ES
    es.index(index='gemini', doc_type='gem', body=msg)
    
    # index account balance to ES
    es.index(index='gemini', doc_type='gem',body=get_btc_balance(con))
    
    # index ticker data to ES
    es.index(index="gemini", doc_type='gem', body=ticker)
    
    if side == 'sell':
        balance = get_btc_balance(con)['amount']
        if balance > 0:
            order = make_order(con, balance, 'sell', ticker)
            score = score_trade(es, order, balance)
            print('Last trade yieled ${} in profit.'.format(score['profit']))

            # index sell order data to ES
            es.index(index="gemini", doc_type='gem', body=order)
            
            # index score to ES
            es.index(index="gemini", doc_type='gem', body=score)
        else:
            print('No balance to sell.')
        return ticker['last'], False

    else: # buy side

        # only trade if we didn't just trade
        if not traded:
            if msg['value'] > threshold[0] and msg['value'] < threshold[1]:
                trade_amount = round((get_usd_balance(con)['available'] * trade_pct) / get_ticker(con)['last'],4)
                print(trade_amount)

                # execute order
                order = make_order(con, trade_amount, side, ticker)

                # index order data to ES
                es.index(index="gemini", doc_type='gem', body=order)

                # print order
                order.pop('timestamp')
                print(json.dumps(order, sort_keys=True,
                    indent=4, separators=(',', ': ')))
                print('\n')

                return ticker['last'], True
            else:
                return last_price, False
        else:
            return last_price, True

In [60]:
# get last price for keeping track of performance
last_price = get_ticker(con)['last']
traded = False

for message in consumer:
    try:
        print(message.value)
        last_price,traded = process_msg(message, fcst, con, es, threshold, trade_pct, last_price, traded)
        print(last_price,traded)
    except Exception as e:
        print("Error occurred: {}".format(e))

{'type': 'ask', 'value': 14037078.934432222, 'timestamp': 1602769807}


/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


11411.44 False
{'type': 'bid', 'value': 14952724.876301888, 'timestamp': 1602769807}
11411.44 False
{'type': 'price_volume', 'value': 1.0652305188384767, 'timestamp': 1602769807}
56.0432
{
    "avg_execution_price": 11408.59,
    "doc_type": "order",
    "exchange": "gemini",
    "executed_amount": 0.395168,
    "id": "659834884",
    "is_cancelled": true,
    "is_hidden": false,
    "is_live": false,
    "options": [
        "immediate-or-cancel"
    ],
    "order_id": "659834884",
    "original_amount": 56.0432,
    "price": 11408.59,
    "reason": "ImmediateOrCancelWouldPost",
    "remaining_amount": 55.648032,
    "side": "buy",
    "symbol": "btcusd",
    "timestampms": 1602770477395,
    "type": "exchange limit",
    "was_forced": false
}


11411.44 True
{'type': 'ask', 'value': 2006407.6115192228, 'timestamp': 1602769867}
11411.44 True
{'type': 'bid', 'value': 2225624.339742532, 'timestamp': 1602769867}
11411.44 True
{'type': 'price_volume', 'value': 1.1092583216713983, 'timesta

KeyboardInterrupt: ignored

In [69]:
query = {
     "query" : {
        "match_all" : {}
    }
}

In [70]:
es.search(body=query, index='gemini', doc_type='gem')

/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'GRmELHUBB_guKLRhUwZ6',
    '_index': 'gemini',
    '_score': 1.0,
    '_source': {'doc_type': 'ask',
     'timestamp': '2020-10-15T13:46:05',
     'value': 10866849.443913555},
    '_type': 'gem'},
   {'_id': 'GhmELHUBB_guKLRhVQab',
    '_index': 'gemini',
    '_score': 1.0,
    '_source': {'amount': 895.58703,
     'available': 895.58703,
     'availableForWithdrawal': 895.58703,
     'currency': 'BTC',
     'doc_type': 'balance',
     'type': 'exchange'},
    '_type': 'gem'},
   {'_id': 'GxmELHUBB_guKLRhVQbh',
    '_index': 'gemini',
    '_score': 1.0,
    '_source': {'ask': 11401.98,
     'bid': 11401.97,
     'doc_type': 'ticker',
     'last': 11395.0,
     'timestamp': '2020-10-15T13:45:00',
     'volume_BTC': 1077.1535584,
     'volume_USD': 12253469.781055884},
    '_type': 'gem'},
   {'_id': 'HBmELHUBB_guKLRhVwaO',
    '_index': 'gemini',
    '_score': 1.0,
    '_source': {'doc_typ